# Feature Engineering

In this notebook, the data will be combined and engineered to be used in model training. Features will be one-hot-encoded as well as normalized

In [108]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import matplotlib.pyplot as plt
import os

from datetime import datetime
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import sagemaker
import boto3

#import warnings
#warnings.filterwarnings('ignore')


In [109]:
# read in the json files
portfolio = pd.read_json('../data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/transcript.json', orient='records', lines=True)

## Portfolio data

In [110]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [111]:
# One-Hot-Encode Channels
mlb = MultiLabelBinarizer()
portfolio = portfolio.join(pd.DataFrame(mlb.fit_transform(portfolio.pop('channels')),
                          columns=mlb.classes_,
                          index=portfolio.index))

In [112]:
# One-Hot-Encode offer_type
portfolio_ohe = pd.concat([portfolio.drop(['offer_type'], axis=1), pd.get_dummies(portfolio.offer_type).add_prefix('offer_')], axis=1)

# Rename column so it can be merged with other data tables
portfolio_ohe.rename(columns={'id': 'id_offer'}, inplace=True)

In [113]:
# Normalize columns
portfolio_ohe.reward = MinMaxScaler().fit_transform(portfolio_ohe.reward.values.reshape(-1, 1))
portfolio_ohe.difficulty = MinMaxScaler().fit_transform(portfolio_ohe.difficulty.values.reshape(-1, 1))
portfolio_ohe.duration = MinMaxScaler().fit_transform(portfolio_ohe.duration.values.reshape(-1, 1))

# describe portfolio table
portfolio_ohe.describe()

,reward,difficulty,duration,email,mobile,social,web,offer_bogo,offer_discount,offer_informational
count,10.000000,10.000000,10.000000,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.420000,0.385000,0.500000,1.0,0.900000,0.600000,0.800000,0.400000,0.400000,0.200000
std,0.358391,0.291595,0.331628,0.0,0.316228,0.516398,0.421637,0.516398,0.516398,0.421637
min,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,0.250000,0.285714,1.0,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.400000,0.425000,0.571429,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,0.500000,0.500000,0.571429,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [114]:
# Drop email and reward column as they are not needed
portfolio_ohe = portfolio_ohe.drop(['email','reward'], axis=1)
portfolio_ohe

,difficulty,duration,id_offer,mobile,social,web,offer_bogo,offer_discount,offer_informational
0,0.50,0.571429,ae264e3637204a6fb9bb56bc8210ddfd,1,1,0,1,0,0
1,0.50,0.285714,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,0,0
2,0.00,0.142857,3f207df678b143eea3cee63160fa8bed,1,0,1,0,0,1
3,0.25,0.571429,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,1,1,0,0
4,1.00,1.000000,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0,1,0,1,0
5,0.35,0.571429,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,0,1,0
6,0.50,1.000000,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,0,1,0
7,0.00,0.000000,5a8bc65990b245e5a138643cd4eb9837,1,1,0,0,0,1
8,0.25,0.285714,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,0,0
9,0.50,0.571429,2906b810c7d4411798c6938adc9daaa5,1,0,1,0,1,0


## Profile data

In [115]:
profile

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [116]:
# Drop na rows
profile.dropna(inplace=True)

In [117]:
# Create groups for age and income
profile['age_bins'] = pd.cut(x=profile.age, bins=[20, 29, 39, 49, 59, 69, 79, 89, 99, 109])
profile['income_bins'] = pd.cut(x=profile.age, bins=[30000, 60000, 90000, 120000])

# Calculate membership of each user in total days
profile['membership_total_days'] = profile.became_member_on.apply(lambda dt: (datetime.today() - datetime.strptime(str(dt), "%Y%m%d")).days)

In [118]:
# One-Hot-Encode gender, age, and income
gender_ohe = pd.get_dummies(profile.gender).add_prefix('gender_')
age_ohe = pd.get_dummies(profile.age_bins).add_prefix('age_')
income_ohe = pd.get_dummies(profile.income_bins).add_prefix('income_')

In [119]:
profile_ohe = pd.concat([profile.drop(['age','age_bins','income','income_bins','became_member_on','gender'], axis=1), gender_ohe, age_ohe, income_ohe], axis=1)

# Rename column so it can be merged with other data tables
profile_ohe.rename(columns={'id': 'id_customer'}, inplace=True)

In [120]:
profile_ohe

,id_customer,membership_total_days,gender_F,gender_M,gender_O,"age_(20, 29]","age_(29, 39]","age_(39, 49]","age_(49, 59]","age_(59, 69]","age_(69, 79]","age_(79, 89]","age_(89, 99]","age_(99, 109]","income_(30000, 60000]","income_(60000, 90000]","income_(90000, 120000]"
1,0610b486422d4921ae7d2bf64640c50b,1518,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,78afa995795e4d85b5d9ceeca43f5fef,1585,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,e2127556f4f64592b11af22de27a7932,1233,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
8,389bc3fa690240e798340f5a15918d5c,1309,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
12,2eeac8d8feae4a8cad5a6af0499a211d,1399,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,6d5f3a774f3d4714ab0c092238f3a1d7,1194,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16996,2cb4f97358b841b9a9773a7aa05a9d77,1155,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
16997,01d26f638c274aa0b965d24cefe3183f,1688,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
16998,9dc1421481194dcd9400aec7c9ae6366,2013,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


## Transcript data

In [121]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [122]:
print(transcript.groupby('event').person.count())
# delete all with transactions
transcript = transcript[transcript.event != 'transaction']
transcript.shape

event
offer completed     33579
offer received      76277
offer viewed        57725
transaction        138953
Name: person, dtype: int64


(167581, 4)

In [123]:
# One-Hot-Encode events
event_ohe = pd.get_dummies(transcript.event).add_prefix('event_')

In [124]:
transcript_ohe = pd.concat([transcript.drop(['event'], axis=1), event_ohe], axis=1)
transcript_ohe = transcript_ohe.drop('value', 1).assign(**pd.DataFrame(transcript_ohe.value.values.tolist()))

# Rename column so it can be merged with other data tables
transcript_ohe.rename(columns={'person': 'id_customer'}, inplace=True)
transcript_ohe.rename(columns={'offer id': 'id_offer'}, inplace=True)

In [125]:
transcript_ohe

,id_customer,time,event_offer completed,event_offer received,event_offer viewed,id_offer,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
...,...,...,...,...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,714,1,0,0,NaN,NaN,NaN
306506,b895c57e8cd047a8872ce02aa54759d6,714,1,0,0,NaN,NaN,NaN
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,714,0,0,1,NaN,NaN,NaN
306509,8431c16f8e1d440880db371a68f82dd0,714,1,0,0,NaN,NaN,NaN


# Combine data

In [170]:
# Combine data tables based on id_offer and id_customer
combined = pd.merge(transcript_ohe, portfolio_ohe, how='left', on="id_offer")
combined = pd.merge(combined, profile_ohe, how='left', on="id_customer")

# Remove columns that are not needed
combined = combined.drop(['id_customer','id_offer','offer_id','offer_informational','time','reward'], axis=1)

# There are likely to be duplicated we do not need.
print(combined.shape)
combined.drop_duplicates(inplace=True)
combined = combined.dropna()
print(combined.shape)

(167581, 26)
(59046, 26)


In [173]:
combined.columns

Index(['event_offer completed', 'event_offer received', 'event_offer viewed',
       'difficulty', 'duration', 'mobile', 'social', 'web', 'offer_bogo',
       'offer_discount', 'membership_total_days', 'gender_F', 'gender_M',
       'gender_O', 'age_(20, 29]', 'age_(29, 39]', 'age_(39, 49]',
       'age_(49, 59]', 'age_(59, 69]', 'age_(69, 79]', 'age_(79, 89]',
       'age_(89, 99]', 'age_(99, 109]', 'income_(30000, 60000]',
       'income_(60000, 90000]', 'income_(90000, 120000]'],
      dtype='object')

In [174]:
print(f"received: {combined['event_offer received'].sum()}")
print(f"received: {combined['event_offer completed'].sum()}")
print(f"received: {combined['event_offer viewed'].sum()}")

received: 30784
received: 10308
received: 17954


In [175]:
successfull_percent = round(combined['event_offer completed'].sum() / combined['event_offer completed'].count(), 4)*100
print(f"total #: {combined['event_offer completed'].sum()}")
print(f"{successfull_percent}% successful offers.")

total #: 10308
17.46% successful offers.


In [176]:
train, test = train_test_split(combined, test_size=0.2)

In [177]:
print(f'Length train: {len(train)}')
print(f'Length test:  {len(test)}')

Length train: 47236
Length test:  11810


#### Save data locally

In [178]:
# Delete any existing data
data_dir = "../formatted_data/"

!rm -rf data_dir

if not os.path.exists("../formatted_data"):
    os.mkdir("../formatted_data")

test.to_csv("../formatted_data/test.csv", index=False)
train.to_csv("../formatted_data/train.csv", index=False)
combined.to_csv("../formatted_data/combined.csv", index=False)

#### save on aws S3

In [179]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/capstone-project'

role = sagemaker.get_execution_role()

In [180]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [181]:
print(input_data)

s3://sagemaker-eu-central-1-855372857567/sagemaker/capstone-project


In [182]:
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

sagemaker/capstone-project/combined.csv
sagemaker/capstone-project/test.csv
sagemaker/capstone-project/train.csv


In [183]:
# Delete bucket if desired
#bucket_to_delete = boto3.resource('s3').Bucket(bucket)
#bucket_to_delete.objects.all().delete()